In [0]:
mid_train = 2204
mid_training =[]

for i in range(mid_train):
  mid_training.append('part_B_final/train_data/images/IMG_'+str(i+1)+'.tiff')

In [0]:
import sys
import os

import warnings




import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms

import numpy as np
import argparse
import json
import cv2
import dataset
import time

In [0]:
import torch.nn as nn
import torch
from torch.nn import functional as F
from torchvision import models

class ContextualModule(nn.Module):
    def __init__(self, features, out_features=512, sizes=(1, 2, 3, 6)):
        super(ContextualModule, self).__init__()
        self.scales = []
        self.scales = nn.ModuleList([self._make_scale(features, size) for size in sizes])
        self.bottleneck = nn.Conv2d(features * 2, out_features, kernel_size=1)
        self.relu = nn.ReLU()
        self.weight_net = nn.Conv2d(features,features,kernel_size=1)

    def __make_weight(self,feature,scale_feature):
        weight_feature = feature - scale_feature
        return F.sigmoid(self.weight_net(weight_feature))

    def _make_scale(self, features, size):
        prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
        conv = nn.Conv2d(features, features, kernel_size=1, bias=False)
        return nn.Sequential(prior, conv)

    def forward(self, feats):
        h, w = feats.size(2), feats.size(3)
        multi_scales = [F.upsample(input=stage(feats), size=(h, w), mode='bilinear') for stage in self.scales]
        weights = [self.__make_weight(feats,scale_feature) for scale_feature in multi_scales]
        overall_features = [(multi_scales[0]*weights[0]+multi_scales[1]*weights[1]+multi_scales[2]*weights[2]+multi_scales[3]*weights[3])/(weights[0]+weights[1]+weights[2]+weights[3])]+ [feats]
        bottle = self.bottleneck(torch.cat(overall_features, 1))
        return self.relu(bottle)

class CANNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CANNet, self).__init__()
        self.seen = 0
        self.context = ContextualModule(512, 512)
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,batch_norm=True, dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
            for i in xrange(len(self.frontend.state_dict().items())):
                self.frontend.state_dict().items()[i][1].data[:] = mod.state_dict().items()[i][1].data[:]

    def forward(self,x):
        x = self.frontend(x)
        x = self.context(x)
        x = self.backend(x)
        x = self.output_layer(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

In [10]:
model = CANNet()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [12]:

checkpoint = torch.load('ALL_WEIGHTS/checkpoint_mid_separate.tar')

model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [0]:


import h5py
import PIL.Image as Image
import numpy as np
import os
import glob
import scipy
from image import *
from model import CANNet
import torch
from torch.autograd import Variable

from sklearn.metrics import mean_squared_error,mean_absolute_error

from torchvision import transforms

In [0]:
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [0]:
model.eval()

names = []
gt = []

In [0]:
#Compute gt for the images
for i in xrange(len(mid_training)):
    img = transform(Image.open(mid_training[i]).convert('RGB')).cuda()
    pure_name = os.path.splitext(os.path.basename(mid_training[i]))[0]
    gt_file = h5py.File(mid_training[i].replace('.tiff','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    #pred_sum = density_1.sum()+density_2.sum()+density_3.sum()+density_4.sum()
    names.append(pure_name)
    #round or int give same values
    gt.append(round(np.sum(groundtruth)))

In [0]:
arrgt = np.asarray(gt)
arrnames = np.asarray(names)

In [0]:
print(arrgt.shape)
print(arrnames.shape)

In [0]:
print(arrgt)
print(arrnames)

In [0]:

class featureExtractor(nn.Module):
    def __init__(self):
        super(featureExtractor, self).__init__()
        self.context = model.context
        self.frontend = model.frontend

    def forward(self,x):
        x = self.frontend(x)
        x = self.context(x)
        return x



net_conv1 = featureExtractor()


In [0]:
net_conv1 = net_conv1.cuda()

In [0]:
features_savepath = 'BEST_TRAIN_MID/' 
if not os.path.exists(features_savepath):
    os.makedirs(features_savepath)

In [0]:
np.save(features_savepath +'/ground_truth_counts_train_mid.npy', arrgt)
np.save(features_savepath +'/ground_truth_names_train_mid.npy', arrnames)

In [29]:
x =[]
y = []
for i in xrange(len(mid_training)):
    img = transform(Image.open(mid_training[i]).convert('RGB')).cuda()
    img = img.unsqueeze(0)
    h,w = img.shape[2:4]
    density= net_conv1(img).data.cpu().numpy()
    act = np.squeeze(density)
    pure_name = os.path.splitext(os.path.basename(mid_training[i]))[0]
    x.append(act)
    y.append(pure_name)

    #density = np.squeeze(3)
    



/usr/local/lib/python2.7/dist-packages/torch/nn/functional.py:2416: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python2.7/dist-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python2.7/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [0]:
s= 0
a = 0
all_feats =[]

In [0]:
for i in range(mid_train):
  feats = []
  for j in range(512):
    s = 0
    for k in range (42):
      for l in range(42):
        s = s + x[i][j][k][l]
    a = s/1764
    feats.append(a)
  all_feats.append(feats)

In [0]:
f = np.array(all_feats)

In [0]:
print(f.shape)

In [0]:
features = 'BEST_TRAIN_MID/FEATURES/' 
if not os.path.exists(features):
    os.makedirs(features)

In [0]:
for i in range(mid_train):
  x=np.array(f[i])
  np.save(features+'/IMG_'+str(i+1)+'.npy',x)